#### Imports

In [1]:
import pandas as pd
import requests
import time
import os
import random

from bs4 import BeautifulSoup

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium import webdriver

from dotenv import load_dotenv
load_dotenv()

True

#### Scrapping LinkedIn Jobs

In [17]:
url = 'https://www.linkedin.com'
driver = webdriver.Chrome()
driver.get(url)

time.sleep(2)

email_box = driver.find_element(By.ID, "session_key")
password_box = driver.find_element(By.ID, "session_password")

# 2. Declare variables
email = os.getenv("email")
password = os.getenv("password")

# 3. Send the text
email_box.send_keys(email)
password_box.send_keys(password)

# 4. Hit enter
password_box.send_keys(Keys.RETURN)

time.sleep(20)
# Navigate to jobs tab
jobs_button = driver.find_element(By.XPATH, '/html/body/div[6]/header/div/nav/ul/li[3]/a')
jobs_button.click()

time.sleep(2)
# Filter for job title
job_box = driver.find_element(By.XPATH, '/html/body/div[6]/header/div/div/div/div[2]/div[2]/div/div/input[1]')
time.sleep(1)
job_box.send_keys('Ciencia de datos')
job_box.send_keys(Keys.ENTER)

time.sleep(2)
# Filter for location
location_box = driver.find_element(By.XPATH, '/html/body/div[6]/header/div/div/div/div[2]/div[2]/div/div/input[1]')
time.sleep(1)
location_box.clear()
time.sleep(3)
location_box.send_keys('Estados Unidos')
location_box.send_keys(Keys.ENTER)

# Save current link
current_link = driver.current_url

def write_file(el):
    job_id = el.get('data-job-id')

    # Open the web page
    driver.get(url + el.find('a').get('href'))
    time.sleep(3)

    html = driver.execute_script("return document.body.outerHTML;") # Javascript script
    soup = BeautifulSoup(html, 'html.parser')

    file_path = f'../data/jobs_html/{job_id}.txt'
    with open(file_path, 'w') as file:
        file.write(str(soup))
    time.sleep(3)

def create_html_files(list_):
    for i in list_[2:4]:
        write_file(i)

# Job scrapping with bs4
html = driver.execute_script("return document.body.outerHTML;")
soup = BeautifulSoup(html, 'html.parser')

pagination = soup.find_all('ul', {'class': 'artdeco-pagination__pages--number'})
pagination = pagination[0].find_all('button')[:8]

for btn in pagination:
    for i in range(5):  # You can adjust the range as needed
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    aria = btn.get("aria-label")
    button = driver.find_element(By.XPATH, f'//button[@aria-label="{aria}"]')
    button.click()
    
    current_link = driver.current_url
    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, 'html.parser')

    time.sleep(3)
    container = soup.find_all('ul', {'class': 'scaffold-layout__list-container'})
    container = container[0]
    list_ = container.find_all('div', {'class': 'job-card-container'})
    create_html_files(list_)
    driver.get(current_link)
    container = []
    list_ = []
    time.sleep(3)


In [13]:
soup = BeautifulSoup(html, 'html.parser')
pagination = soup.find_all('ul', {'class': 'artdeco-pagination__pages--number'})
pagination = pagination[0].find_all('button')[:8]
pagination

container = soup.find_all('ul', {'class': 'scaffold-layout__list-container'})
container = container[0]
with open('../data/job_lists.txt', 'w') as file:
    file.write(str(container))
    
list_ = container.find_all('div', {'class': 'job-card-container'})
list_[]

IndexError: list index out of range

In [67]:
def get_job_card_info(el):
    print(el.find('a').get('href'))
    job = {}
    job['job_id'] = el.get('data-job-id')
    job['job_title'] = el.find('a').getText().strip()
    job['company'] = el.find_all('span', {'class': 'job-card-container__primary-description'})[0].getText().strip()
    job['location'] = el.find_all('li', {'class': 'job-card-container__metadata-item'})[0].getText().strip()
    time.sleep(5)
    return job

def build_linkedin_dataset(list_):
    jobs = []
    for card in list_[1:5]:
        jobs.append(get_job_card_info(card))
    print(jobs)

list_ = container.find_all('div', {'class': 'job-card-container'})
list_
jobs = build_linkedin_dataset(list_)

/jobs/view/3769019036/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=p8zqT9JCgLPTFHag5H5ivQ%3D%3D&trackingId=uZ%2FXzNImMNk05AEvUAttaQ%3D%3D&trk=flagship3_search_srp_jobs
/jobs/view/3769408548/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=p8zqT9JCgLPTFHag5H5ivQ%3D%3D&trackingId=0VSdJOBbt96ps%2BTS%2Fq2J%2Fg%3D%3D&trk=flagship3_search_srp_jobs
/jobs/view/3774942158/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=p8zqT9JCgLPTFHag5H5ivQ%3D%3D&trackingId=W0%2BOVPzEC83G1lG7w1kL3Q%3D%3D&trk=flagship3_search_srp_jobs
/jobs/view/3772973391/?eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=p8zqT9JCgLPTFHag5H5ivQ%3D%3D&trackingId=5aOXCke%2BP2YCzLerKHWcFg%3D%3D&trk=flagship3_search_srp_jobs
[{'job_id': '3769019036', 'job_title': 'Junior Data Analyst', 'company': 'Coders Data', 'location': 'California, Estados Unidos (En remoto)'}, {'job_id': '3769408548', 'job_title': 'Junior Data Analyst', 'company': 'TEKsystems', 'location': 'Honolulú, HI (Híbrido)'}, {'job_id': '3774942158', 'job_title': 'Data Analyst I', 'company': 'Cintas', 'location':